* Pseudo labeling
    * Jeremy made a batch iterator class that combines training data with test data that you labeled with a previous model.
    * You can also add validation data, at the end of training to improve the last bit.
* Embeddings
    * embeddings() just looks up the embeddings of a certain called user/movie index. Then you merge() them with a dot product, add biases and compare to the actual rating to find the error.
    * 8:30 Jeremy goes trough the process of the model in excel
    * read word2vec paper and check differences between different sizes of wordlist/factors, people already did this.
* RNN's
    * The purpose of RNN's is really all about memory. memory of a sentence, syntax, quotes, etc.
    * 3 char model, making a neural net that guesses the 4th character after training on a 4 char parts chopped sequence of the entire Nietzsche text, training on the first 3 and having the fourth to calculate loss.
        * Import text, create list of all charachters and list of corresponding indices. convert text into indices as idx.
        * Create model that predicts 4th char from previous three; create lists of 1,5,9... 2,6,10... 3,7,11... en 4,8,12.... Make x1, x2 and x3 the first three, y is the last set. What you will do is train what y is based on x1,x2,x3 for every iteration of 4 characters.
        * for this model you have three kinds of states; input, hidden and output. Also three kinds of operations on these states; input to hidden, hidden to hidden, hidden to output.
        * create an embedding for all three locations. The embedding function returns the input and the embedded input layers.
        * Choose amount of activations n_hidden.
        * The input to hidden operation in the functional api can be made as follows. Create a Dense layer that only has the amount of activations defined so that you can add the state it operates on later. Normally you'd have an operation with a state added to the back. "dense_in = Dense(n_hidden, activation='relu')" and then "c1_hidden = dense_in(c1)".
        * Now you already have the operation to hidden state defined, just use the previous hidden state as your input for the next hidden state. EXCEPT that you want a different activation (tanh instead of relu) so you define a new operation.
        * A layer can be seen as an operation. If the same operation is called multiple times, this can be seen in model.summary() as the first parameter of 'connected to'
        * ACTIVATION FUNCITONS Can you mix them? It's not usual that people do it but you can do it. You can use pretty much any activation functions, some just train quicker such as relu and variants.
    * Our first RNN! The same as before but now with 8 characters. Functional difference is that cs is a variable and orange arrow (hidden to hidden) is initialized as identity. Coding difference is that layers are written in for loops. Predicting char n using chars 1 to n-1 for n = 8.
        * Smart initialization of RNN with relu: use identity matrix as initilization matrix for the recurrent weight matrix. "A simpel way to intialize recurrent networks of rectified linear units".
        * Not much changed, just smart and scalable implementation.
        * this is useful for sentiment analysis, I don't see why.
    * Our first RNN with Keras!
        * The same as before but than as a sequential model
    * Returning sequences.  We are making some changes; we integrate the output into the recurrent part. So: Predicting chars 2 to n using chars 1 to n-1. Benefit is that we have a learning point after every new input char. 
        * Input is the same; lists of 8mod(i) indices for all i = range(8) of the text. 
        * The output is the same as the input but then everything moved by 1 character, the output has the same dimensions as the input (8 per iteration, where every char of the output is the prediction after that char on the same location of the input array). This makes sense because we as many outputs as inputs. 
        * The first input is moved to the recurrent part of the model and the model is initialized with zeros (because zeros don't do shit).
        * In training, you have losses for all the 8 outputs where the 8th loss goes down most because it has more information about the previous stuff whereas the first output only has the information about all previous 1st position chars.
    * Sequence model with Keras. Same as previous implementation of 8 char model but with the new output array and return_sequences=True
        * To have the outputs inside the loop, change return_sequences variable to True. This means that after every new input character, an output is generated.
        * change input dimensionality.
        * use TimeDistributed(Dense()) for Dense() layers.
    * Stateful model with keras. Previously, we trained the embedding of the characters and re-initialized the hidden state for every new sequence. Now we also re-use the hidden state between batches.
        * To understand long term dependencies we can change shuffle=True in model.fit() to shuffle=False because maybe we can get some of the information of the previous sentence in the hidden to hidden layer.
        * We can't re-initialize to zeros after each batch; in keras this means stateful=True (this needs shuffle=False).
        * Theorie: Training stateful models is harder because we re-use the hidden-hidden multiplication everytime we have a new input which make the chances of having exploding gradients/activations because we're using stateful=True. Solution: LSTM, Long Short Term Memory. LSTM has a nn in the hidden-hidden operation that can learn to select which part of the hidden-hidden weight matrix should be used and which part not. More next week.
        * Because of this tendency of a stateful model to explode, Jeremy had to introduce an LSTM and a BatchNormalization() layer.
        * Theory: Traing with shuffle=False takes longer because it's harder to parallelize.
        * Theory: Can't you do batchnorm in the loop? Layernormalizations works with RNN's but lot of discussion. Jeremy's experience is not so well.
    * Best stateful model with keras: char-rnn.ipynb.
        * Sequential model with an LSTM to an LSTM. This means that the output vector of 42 embedding factors is passed as the input for the second LSTM. Deep RNN.
        * Why is LSTM - LSTM useful? the connection between the two rnn's is dense matrix, capturing the difficulty in language.
        * Add dropout inside your RNN
        * Keras: return_sequences=True in LSTM's need an TimeDistributed(Dense()) layer after them instead of normal Dense(). return_sequences=True means that the output is within the recurrent part of the model, so the model outputs after every input that is passed. The state of the previous layer is not a vector of 256 but a vector of 8 times 256 because it's predicting 8 outputs. We can't use a single dense layer because that takes 1d as input. TimeDistributed just creates, in this case, 8 copies of the dense layer with the same weight matrix.
        * Add dropout after the dense layer and another timedes dense layer.
        * Later; add batchnorm to increase training rate.
        * Examples, swiftkey, anomaly detection(!), music
    * Before Theano RNN; One-hot sequence model with keras
        * As a loss function we used sparse_catergorical_entropy previously, this is identical to one-hot-encoded categorical entropy but you don't need to OHE. This is usefull for a lot of output categories ie a word model.
    * Theano RNN, keras is using theano so if you want to get more into detail, you can use Theano.
        * Theano doesn't have layers so we need to compute all the state and weight matrices by hand.
        * Theano is build to describe a model and Theano then transforms it into something that the gpu can compute. First you explain the kinds of variables that you'll later use.
        * Theory: scan is a function that can be parallelized. 
        * Python: using break points: import pdb, somewhere: pdb.set_trace(). You can look around in the current state of the process!!!!!!!
    * Difference between using a CNN or RNN for text is that a CNN is better for small amounts of text whereas an RNN is better for more text.
        